## Experiments (baseline)
This notebook is used for conduct the baseline prediction with the pre-trained LLMs. Adapted from [Large Language Models Are Zero Shot Time Series Forecasters
](https://github.com/ngruver/llmtime)

We predict both univariate and multivariate time series (with each prediction focusing on a single sequence). Additionally, we offer the option to apply Moving Average (MA) for smoothing the input time series.

In [ ]:
from darts import TimeSeries, concatenate
from darts.utils.callbacks import TFMProgressBar
from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller
from darts.metrics import mape, r2_score
from darts.datasets import EnergyDataset
from darts import concatenate


: 

In [ ]:
import time

from statsmodels.datasets import co2
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from statsmodels.tsa.seasonal import STL
import numpy as np
import copy
from sklearn.preprocessing import StandardScaler

import matplotlib.font_manager as fm
import seaborn as sns

In [2]:
import os
os.environ['OMP_NUM_THREADS'] = '4'

from data1.serialize import SerializerSettings
from data1.small_context import get_datasets,get_memorization_datasets

In [ ]:
from models.utils import grid_iter
from models.validation_likelihood_tuning import get_autotuned_predictions_data
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from models import llmtime
from models.llmtime import get_llmtime_predictions_data

In [4]:
import pickle

In [ ]:
import json
import openai

with open(r'config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)
    
openai.api_key = config['OPENAI_API_KEY']
openai.api_base = config['OPENAI_API_BASE']

### Dataset Preparation

In [ ]:
def dataset_obtain(testfrac=0.2):
    '''
    获得 numpy 结构数据集
    '''
    datasets_path = {
    'us_births': 'datasets/monash/us_births.pkl',  # daily
    'saugeenday': 'datasets/monash/saugeenday.pkl'  # daily
    }
    datas = []
    for name, data_path in datasets_path.items():
        with open(data_path, 'rb') as file:
            # 加载数据
            data = pickle.load(file)
        series = pd.Series(np.concatenate((data[0][0][0], data[0][0][1])))
        splitpoint = int(len(series)*(1-testfrac))
        train = series.iloc[:splitpoint]  # Only test the last couples of samples
        test = series.iloc[splitpoint:]
        datas.append((train,test))
    return dict(zip(datasets_path.keys(),datas))

In [ ]:
# Retrieve the seasonal period for a given dataset.
def period_get(name):
    '''
    Returns the known or manually estimated seasonal period for a specified dataset.

    Args:
        name (str): The name of the dataset.

    Returns:
        period (int): The seasonal period length. Default is 365 (e.g., for daily yearly data).
    '''
    period = 365  # Default period (e.g., daily data with yearly seasonality)
    
    if name == "HeartRateDataset":
        period = 144
    if name == "GasRateCO2Dataset":
        period = 39
    if name == "AirPassengersDataset":
        period = 12
    if name == "AusBeerDataset":
        period = 4
    if name == "MonthlyMilkDataset":
        period = 12
    if name == "SunspotsDataset":
        period = 12
    if name == "WineDataset":
        period = 12
    if name == "WoolyDataset":
        period = 4
    if name == "IstanbulTraffic":
        period = 24
    if name == "TurkeyPower":
        period = 7    # Weekly cycle; manually estimated, not from official documentation
    if name == "us_birth":
        period = 7    # Weekly cycle; manually estimated, not from official documentation
    if name == "saugeenday":
        period = 7    # Weekly cycle; manually estimated, not from official documentation

    return period

## Univariate Timeseries

### Data-preprocessing

In [ ]:
def preprocessing_data(train, test):
    train_reshaped = train.values.reshape(-1, 1)
    test_reshaped = test.values.reshape(-1, 1)
    scaler = StandardScaler()
    scaler.fit(train_reshaped)
    standardized_train = scaler.transform(train_reshaped)
    train = pd.Series(standardized_train.flatten(), index=train.index)
    standardized_test = scaler.transform(test_reshaped)
    test = pd.Series(standardized_test.flatten(), index=test.index)    
    return train, test

In [ ]:
datasets = get_datasets()  
datasets_tmp = get_memorization_datasets()
datasets.update(datasets_tmp)
datasets_tmp = dataset_obtain()
datasets.update(datasets_tmp)

for name in datasets.keys():
    train, test = datasets[name]
    train, test = preprocessing_data(train, test)
    dataset = (train, test)
    datasets[name] = dataset


### Forecasting

In [ ]:
gpt4_hypers = dict(
    alpha=0.3,
    basic=True,
    temp=1.0,
    top_p=0.8,
    settings=SerializerSettings(base=10, prec=3, signed=True, time_sep=', ', bit_sep='', minus_sign='-')
)

gpt3_hypers = dict(
    temp=0.7,
    alpha=0.95,
    beta=0.3,
    basic=False,
    settings=SerializerSettings(base=10, prec=3, signed=True, half_bin_correction=True)
)

promptcast_hypers = dict(
    temp=0.7,
    settings=SerializerSettings(base=10, prec=0, signed=True,
                                time_sep=', ',
                                bit_sep='',
                                plus_sign='',
                                minus_sign='-',
                                half_bin_correction=False,
                                decimal_point='')
)


gemini_hypers = {
    'temp': 0.2,  # recommended: [0.2, 0.4]
    'alpha': 0.95,
    'beta': 0.3,
    'basic': [False],
    'settings': [SerializerSettings(base=10, prec=3, signed=True,half_bin_correction=True)],  # recommended: prec=3
}

In [ ]:
model_hypers = {
    'LLMTime GPT-3.5': {'model': 'gpt-3.5-turbo-instruct', **gpt3_hypers},
    # 'LLMTime GPT-4': {'model': 'gpt-4', **gpt4_hypers},
    # 'LLMTime GPT-3': {'model': 'text-davinci-003', **gpt3_hypers},
    # 'PromptCast GPT-3': {'model': 'text-davinci-003', **promptcast_hypers},
    # 'ARIMA': arima_hypers,
    'gemini-1.0-pro': {'model': 'gemini-1.0-pro', **gemini_hypers},
}

model_predict_fns = {
    'LLMTime GPT-3.5': get_llmtime_predictions_data,
    # 'LLMTime GPT-4': get_llmtime_predictions_data,
    # 'PromptCast GPT-3': get_promptcast_predictions_data,
    # 'ARIMA': get_arima_predictions_data,
    'gemini-1.0-pro': get_llmtime_predictions_data, 
}

In [32]:
model_names = list(model_predict_fns.keys())

### Datasets(Darts and Monash_Not commonly used datasets)

In [33]:
# Selected datasets
# name_list = ['GasRateCO2Dataset', 'WineDataset', 'SunspotsDataset', 'AusBeerDataset']

# only for partial test 
name_list = ['AirPassengersDataset', 'AusBeerDataset', 'GasRateCO2Dataset', 'MonthlyMilkDataset', 'SunspotsDataset', 'WineDataset', 'WoolyDataset', 'HeartRateDataset', 'IstanbulTraffic', 'TSMCStock', 'TurkeyPower', 'us_births', 'saugeenday']

In [ ]:
def MA_process(name, train, test, test_length=96):
    period = period_get(name)
    data_tmp = pd.concat([train, test])
    out = data_tmp.rolling(window=period).mean().dropna()
    train_tmp = out[:-test_length]
    test_tmp = out[-test_length:]
    return train_tmp, test_tmp

def metrics_computation(test, median_pred):
    mse = mean_squared_error(test, median_pred)
    mae = mean_absolute_error(test, median_pred)
    mape = mean_absolute_percentage_error(test, median_pred) * 100
    r2 = r2_score(test, median_pred)        
    return mse, mae, mape, r2

def MA_recover(data_pred, median_pred, record_tmp, record_list_tmp, period=8, whether_MA=False):
    data_pred_tmp = copy.deepcopy(data_pred)
    median_pred_tmp = copy.deepcopy(median_pred)
    if whether_MA is True:
        for index in range(data_pred.shape[1]):
            if index >= period:
                data_pred.iloc[:, index] = period * (data_pred_tmp.iloc[:, index]-data_pred_tmp.iloc[:, index-1])+data_pred.iloc[:, index-period]
                median_pred.iloc[index] = period * (median_pred_tmp.iloc[index]-median_pred_tmp.iloc[index-1])+median_pred.iloc[index-period]
            else:
                if index == 0:
                    data_pred.iloc[:, index] = period * (data_pred_tmp.iloc[:, index]-record_tmp)+record_list_tmp.iloc[index]
                    median_pred.iloc[index] = period * (median_pred_tmp.iloc[index]-record_tmp)+record_list_tmp.iloc[index] 
                else:
                    data_pred.iloc[:, index] = period * (data_pred_tmp.iloc[:, index]-data_pred_tmp.iloc[:, index-1])+record_list_tmp.iloc[index]
                    median_pred.iloc[index] = period * (median_pred_tmp.iloc[index]-median_pred_tmp.iloc[index-1])+record_list_tmp.iloc[index]
    return data_pred, median_pred
 

In [ ]:
def forecasting_w_LLMs(datasets, name_list, model_names, whether_MA=False, pred_period=8, whether_multi=False):
    
    for name, dataset in datasets.items():
        if name not in name_list:  
            continue
    
        data = datasets[name]
        train, test = data  

        if name in ['us_births', 'saugeenday']:  
            train = train.iloc[-3080:]  
            test = test.iloc[:96]  
        
        test_original = copy.deepcopy(test)
        record_list_tmp = train[-pred_period:]
        
        if whether_MA:
            train, test = MA_process(name=name, train=train, test=test, test_length=len(test))
        record_tmp = train.iloc[-1]
    
        for model in model_names: 
            print(model)
            if model == 'gemini-1.0-pro':
                whether_blank = False
            else:
                whether_blank = True
            model_hypers[model].update({'dataset_name': name})  
            hypers = list(grid_iter(model_hypers[model]))  
            num_samples = 10  
            
            print(type(train))
            pred_dict = get_autotuned_predictions_data(train, test, hypers, num_samples, model_predict_fns[model], verbose=False, parallel=False, whether_blanket=whether_blank, whether_multi=whether_multi) 
            data_pred = pred_dict['samples']  
            median_pred = pred_dict['median'] 
            
            data_pred, median_pred = MA_recover(data_pred=data_pred, median_pred=median_pred, record_tmp=record_tmp, record_list_tmp=record_list_tmp, period=pred_period, whether_MA=whether_MA)
                            
            pred_dict['samples'] = data_pred  
            pred_dict['median'] = median_pred
    
            mse, mae, mape, r2 = metrics_computation(test_original, median_pred)


In [ ]:
whether_MA = False
forecasting_w_LLMs(datasets, name_list, model_names, whether_MA=whether_MA)


#### with MA

In [ ]:
whether_MA = True
forecasting_w_LLMs(datasets, name_list, model_names, whether_MA=whether_MA)

## Multi-variate datasets

In [82]:
def get_datasets_multi(n=-1, testfrac=0.2):
    datasets = [
        'electricity',
        'ETTh1',  
        'ETTh2',
        'ETTm1',
        'ETTm2', 
        'exchange_rate',
        'national_illness',
        'traffic',
        'weather'
    ]
    datas = []
    for i, dsname in enumerate(datasets):
        path = f'datasets/multi_variate_datasets/{dsname}.csv'
        series = pd.read_csv(path)
        splitpoint = int(series.shape[0]*(1-testfrac))
        train = series.iloc[:splitpoint]  # Only test the last couples of samples
        test = series.iloc[splitpoint:]
        datas.append((train,test))
        if i+1==n:
            break
    return dict(zip(datasets,datas))

In [83]:
datasets_tmp = get_datasets_multi()

#### Predict only 1 sequence each time

In [ ]:
pred_dict_list = []

for name, dataset in datasets_tmp.items():
    def metrics_computation(test, median_pred):
        mse = mean_squared_error(test, median_pred)
        mae = mean_absolute_error(test, median_pred)
        mape = mean_absolute_percentage_error(test, median_pred) * 100
        r2 = r2_score(test, median_pred)        
        return mse, mae, mape, r2
        
    train, test = dataset 
    train = train[len(train)-1024:]['OT']  # We here cutoff 1024 numbers for the training set (in terms of context length)
    test = test[:96]['OT']  # next 96 numbers
    
    for model in model_names: 
        model_hypers[model].update({'dataset_name': name})  # for promptcast, dict 添加元素
        hypers = list(grid_iter(model_hypers[model]))  # 简单理解为传参
        num_samples = 10  # 表示重复的预测的次数, 原本需要为10, 此处暂时选择 2 来进行测试
        pred_dict = get_autotuned_predictions_data(train, test, hypers, num_samples, model_predict_fns[model], verbose=False, parallel=False, whether_blanket=False) 
        pred_dict_list.append(pred_dict)

Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gpt-4', 'alpha': 0.3, 'basic': True, 'temp': 1.0, 'top_p': 0.8, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=', ', bit_sep='', plus_sign='', minus_sign='-', half_bin_correction=True, decimal_point='', missing_str=' Nan'), 'dataset_name': 'electricity'}) 
 with NLL inf




  0%|          | 0/1 [00:00<?, ?it/s]

943, 951, 970, 1018, 1075, 1171, 1250, 1226, 1231, 1307, 1274, 1313, 1410, 1450, 1501, 1421, 1306, 1284, 1204, 1151, 1121, 1086, 1063, 1037, 1016, 996, 1020, 961, 1030, 1123, 1186, 1198, 1320, 1337, 1376, 1384, 1369, 1408, 1459, 1385, 1334, 1301, 1141, 1167, 1109, 1025, 1049, 1009, 1007, 975, 1003, 936, 1046, 1158, 1180, 1237, 1266, 1307, 1305, 1361, 1457, 1528, 1488, 1324, 1259, 1261, 1181, 1154, 1136, 1101, 1048, 1055, 1023, 993, 970, 975, 1059, 1163, 1204, 1256, 1329, 1353, 1377, 1542, 1575, 1661, 1772, 1610, 1512, 1364, 1242, 1182, 1147, 1130, 1105, 1069, 1051, 1046, 1049, 1038, 1078, 1131, 1151, 1191, 1203, 1156, 1150, 1179, 1215, 1242, 1216, 1249, 1239, 1176, 1124, 1119, 1077, 1035, 1045, 1029, 1023, 1015, 982, 998, 936, 986, 964, 1031, 1034, 1010, 1024, 1068, 1036, 1080, 1050, 1021, 987, 999, 951, 973, 920, 937, 887, 923, 874, 915, 898, 913, 734, 1013, 952, 937, 992, 988, 944, 1005, 957, 990, 953, 950, 970, 945, 912, 950, 884, 909, 912, 899, 887, 890, 890, 972, 985, 1095, 1112, 



100%|██████████| 1/1 [00:17<00:00, 17.81s/it]


Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gpt-4', 'alpha': 0.3, 'basic': True, 'temp': 1.0, 'top_p': 0.8, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=', ', bit_sep='', plus_sign='', minus_sign='-', half_bin_correction=True, decimal_point='', missing_str=' Nan'), 'dataset_name': 'ETTh1'}) 
 with NLL inf


...



100%|██████████| 1/1 [00:20<00:00, 20.05s/it]


Sampling with best hyper... defaultdict(<class 'dict'>, {'model': 'gpt-4', 'alpha': 0.3, 'basic': True, 'temp': 1.0, 'top_p': 0.8, 'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=', ', bit_sep='', plus_sign='', minus_sign='-', half_bin_correction=True, decimal_point='', missing_str=' Nan'), 'dataset_name': 'weather'}) 
 with NLL inf




  0%|          | 0/1 [00:00<?, ?it/s]

987, 986, 984, 985, 987, 987, 987, 986, 996, 1001, 996, 986, 990, 1008, 1015, 1006, 1010, 1007, 1005, 999, 1004, 1032, 1017, 1008, 1010, 1012, 1014, 1023, 1019, 1016, 1028, 1025, 1034, 1033, 1039, 1036, 1034, 1044, 1055, 1064, 1064, 1074, 1075, 1070, 1072, 1067, 1059, 1053, 1057, 1054, 1059, 1080, 1086, 1080, 1077, 1085, 1075, 1078, 1066, 1068, 1073, 1072, 1079, 1082, 1081, 1074, 1072, 1075, 1074, 1075, 1092, 1080, 1078, 1081, 1089, 1099, 1097, 1101, 1099, 1094, 1092, 1086, 1087, 1093, 1084, 1080, 1092, 1099, 1093, 1107, 1103, 1113, 1114, 1108, 1116, 1120, 1124, 1113, 1115, 1120, 1120, 1114, 1095, 1086, 1045, 1027, 1034, 1037, 1035, 1025, 1018, 1020, 1018, 1013, 1013, 1011, 1001, 1001, 1005, 1006, 1007, 1005, 1005, 1000, 998, 999, 996, 996, 996, 997, 1006, 1006, 1006, 1007, 1007, 1005, 1006, 1002, 1001, 998, 993, 988, 989, 990, 990, 993, 996, 997, 997, 993, 990, 991, 993, 992, 992, 994, 1001, 998, 998, 999, 1008, 1016, 1004, 1001, 1000, 999, 1000, 1003, 1005, 1005, 1005, 1005, 1007, 10



100%|██████████| 1/1 [00:19<00:00, 19.21s/it]
